In [2]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import os
import shutil
import random
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
%matplotlib inline

Using TensorFlow backend.


In [3]:
path= 'mydata'
labelfile= 'labels.csv'
batch_size_val=50
steps_per_epoch_val=2000
epochs_val=10
imageDimensions=(224,224,3)
testRatio=0.2
validationRatio=0.2

In [4]:
count=0
images=[]
classNo=[]
myList= os.listdir(path)
print("total classes detected:",len(myList))
noOfclasses=len(myList)
print('Importing classes')
for x in range(0,noOfclasses):
    myPickList= os.listdir(path+'/'+str(count))
    for y in myPickList:
        curImg=cv2.imread(path+'/'+str(count)+'/'+y)
        images.append(curImg)
        classNo.append(count)
    print(count," ")
    count+=1
print(" ")
images=np.array(images)
classNo=np.array(classNo)

total classes detected: 4
Importing classes
0  
1  
2  
3  
 


In [5]:
x_train, x_test, y_train, y_test= train_test_split(images,classNo,test_size=testRatio)
x_train, x_validation, y_train, y_validation= train_test_split(images,classNo,test_size=validationRatio)

In [7]:
data=pd.read_csv(labelfile)
print("data shape",data.shape,type(data))

data shape (3, 2) <class 'pandas.core.frame.DataFrame'>


In [ ]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
        

plots(images, titles=classNo)
print(images.shape,'\n')

In [ ]:
def preprocessing(img):  
    img = cv2.equalizeHist(img) 
    img = img / 255
    return img 
  
x_train = np.array(list(map(preprocessing, x_train))) 
x_validation = np.array(list(map(preprocessing, x_validation))) 
x_test = np.array(list(map(preprocessing, x_test))) 
  
x_train = x_train.reshape(34799, 32, 32, 1) 
x_validation = x_validation.reshape(4410, 32, 32, 1) 
x_test = x_test.reshape(12630, 32, 32, 1)